## <span style='color:#ff5f27'> 📝 Imports

In [1]:
# !pip install -r requirements.txt --quiet

In [2]:
import joblib

from functions.llm_chain import load_model, get_llm_chain, generate_response

## <span style="color:#ff5f27;"> 🔮 Connect to Hopsworks Feature Store </span>

In [3]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store() 

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://snurran.hops.works/p/5242
Connected. Call `.close()` to terminate connection gracefully.


## <span style="color:#ff5f27;"> ⚙️ Feature View Retrieval</span>

In [4]:
# Retrieve the 'air_quality_fv' feature view
feature_view = fs.get_feature_view(
    name='air_quality_fv',
    version=1,
)

# Initialize batch scoring
feature_view.init_batch_scoring(1)

## <span style="color:#ff5f27;">🪝 Retrieve AirQuality Model from Model Registry</span>

In [5]:
# Retrieve the model registry
mr = project.get_model_registry()

# Retrieve the 'air_quality_xgboost_model' from the model registry
retrieved_model = mr.get_model(
    name="air_quality_xgboost_model",
    version=1,
)

# Download the saved model artifacts to a local directory
saved_model_dir = retrieved_model.download()

Connected. Call `.close()` to terminate connection gracefully.


In [6]:
# Load the XGBoost regressor model and label encoder from the saved model directory
model_air_quality = joblib.load(saved_model_dir + "/xgboost_regressor.pkl")
encoder = joblib.load(saved_model_dir + "/label_encoder.pkl")

# Display the retrieved XGBoost regressor model
model_air_quality

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

## <span style='color:#ff5f27'>⬇️ LLM Loading

In [7]:
# Load the LLM and its corresponding tokenizer.
model_llm, tokenizer = load_model()

tokenizer_config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/101 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

2024-03-27 14:35:05,651 INFO: We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

## <span style='color:#ff5f27'>⛓️ LangChain

In [8]:
# Create and configure a language model chain.
llm_chain = get_llm_chain(
    model_llm,
    tokenizer,
)

## <span style='color:#ff5f27'>🧬 Model Inference


In [9]:
QUESTION7 = "Hi!"

response7 = generate_response(
    QUESTION7,
    feature_view,
    model_air_quality,
    encoder,
    model_llm,
    tokenizer,
    llm_chain,
    verbose=True,
)

print(response7)

🗓️ Today's date: Wednesday, 2024-03-27
📖 

Hello! I can help you with information about the air quality in the city. According to the data I have, on the 27th of March, 2024, the air quality in the city was considered safe for most people. The concentration of PM2.5 was at 12 µg/m³, which is within the safe limit of 25 µg/m³. The concentration of NO2 was at 20 µg/m³, which is also within the safe limit of 40 µg/m³. The concentration of O3 was at 30 µg/m³, which is below the safe limit of 40 µg/m³. 

Based on these readings, it is safe for you to go for a walk or engage in outdoor activities. However, if you have a pre-existing respiratory condition, it is always recommended to consult with your doctor before engaging in outdoor activities.


In [10]:
QUESTION = "Who are you?"

response = generate_response(
    QUESTION,
    feature_view,
    model_air_quality,
    encoder,
    model_llm,
    tokenizer,
    llm_chain,
    verbose=True,
)

print(response)

🗓️ Today's date: Wednesday, 2024-03-27
📖 

I am an expert in air quality, but I'm unable to assist you at the moment.


In [11]:
QUESTION1 = "What was the air quality from 2024-01-10 till 2024-01-14 in New York?"

response1 = generate_response(
    QUESTION1, 
    feature_view, 
    model_air_quality, 
    encoder,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=True,
)

print(response1)

Finished: Reading data from Hopsworks, using ArrowFlight (8.87s) 
🗓️ Today's date: Wednesday, 2024-03-27
📖 Air Quality Measurements for New York:
Date: 2024-01-10; Air Quality: 7.2
Date: 2024-01-11; Air Quality: 5.9
Date: 2024-01-12; Air Quality: 10.8
Date: 2024-01-13; Air Quality: 5.9
Date: 2024-01-14; Air Quality: 5.1

The air quality in New York from January 10th to January 14th was generally within a safe range. The average air quality during this period was 6.9, which indicates good air quality. This is a suitable time for outdoor activities, such as going for a walk or bike ride.


In [12]:
QUESTION11 = "When and what was the maximum air quality from 2024-01-10 till 2024-01-14 in New York?"

response11 = generate_response(
    QUESTION11, 
    feature_view, 
    model_air_quality,
    encoder,
    model_llm,
    tokenizer,
    llm_chain,
    verbose=True,
)

print(response11)

Finished: Reading data from Hopsworks, using ArrowFlight (8.12s) 
🗓️ Today's date: Wednesday, 2024-03-27
📖 Air Quality Measurements for New York:
Date: 2024-01-10; Air Quality: 7.2
Date: 2024-01-11; Air Quality: 5.9
Date: 2024-01-12; Air Quality: 10.8
Date: 2024-01-13; Air Quality: 5.9
Date: 2024-01-14; Air Quality: 5.1

The maximum air quality in New York from 2024-01-10 to 2024-01-14 was on 2024-01-12 with an air quality level of 10.8. This level is considered unhealthy for sensitive groups and may cause breathing difficulties for some individuals. It is advisable to limit outdoor activities on days with such high air quality levels.


In [13]:
QUESTION12 = "When and what was the minimum air quality from 2024-01-10 till 2024-01-14 in New York?"

response12 = generate_response(
    QUESTION12, 
    feature_view,  
    model_air_quality, 
    encoder,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=True,
)

print(response12)

Finished: Reading data from Hopsworks, using ArrowFlight (8.35s) 
🗓️ Today's date: Wednesday, 2024-03-27
📖 Air Quality Measurements for New York:
Date: 2024-01-10; Air Quality: 7.2
Date: 2024-01-11; Air Quality: 5.9
Date: 2024-01-12; Air Quality: 10.8
Date: 2024-01-13; Air Quality: 5.9
Date: 2024-01-14; Air Quality: 5.1

The minimum air quality during that period was on 2024-01-14, with an air quality level of 5.1. This is considered to be good air quality, which means it is safe to go for a walk or engage in outdoor activities.


In [14]:
QUESTION2 = "What was the air quality yesterday in London?"

response2 = generate_response(
    QUESTION2,
    feature_view,
    model_air_quality,
    encoder,
    model_llm,
    tokenizer,
    llm_chain,
    verbose=True,
)

print(response2)

Finished: Reading data from Hopsworks, using ArrowFlight (8.02s) 
🗓️ Today's date: Wednesday, 2024-03-27
📖 Air Quality Measurements for London:
Date: 2024-03-26; Air Quality: 12.7

The air quality in London yesterday, on March 26th, was 12.7. This indicates that the air quality was within the safe range, making it suitable for outdoor activities.


In [15]:
QUESTION = "What was the air quality like last week in London?"

response = generate_response(
    QUESTION,
    feature_view, 
    model_air_quality,
    encoder,
    model_llm,
    tokenizer,
    llm_chain,
    verbose=True,
)

print(response)

Finished: Reading data from Hopsworks, using ArrowFlight (8.49s) 
🗓️ Today's date: Wednesday, 2024-03-27
📖 Air Quality Measurements for London:
Date: 2024-03-18; Air Quality: 12.7
Date: 2024-03-19; Air Quality: 9.7
Date: 2024-03-20; Air Quality: 15.6
Date: 2024-03-21; Air Quality: 16.7
Date: 2024-03-22; Air Quality: 8.7
Date: 2024-03-23; Air Quality: 5.4
Date: 2024-03-24; Air Quality: 6.4

Last week in London, the air quality was generally good. On 2024-03-19, it was slightly polluted with an air quality of 9.7. The air quality improved on 2024-03-20, reaching a moderate level of 15.6. It was quite clean on 2024-03-21 with an air quality of 16.7. However, it became slightly polluted again on 2024-03-22 with an air quality of 8.7. The air quality improved significantly on 2024-03-23, reaching a healthy level of 5.4. On 2024-03-24, the air quality was slightly better than the previous day with an air quality of 6.4. Overall, the air quality last week in London was mostly good with some s

In [16]:
QUESTION3 = "What will the air quality be like in London in 2024-04-02?"

response3 = generate_response(
    QUESTION3, 
    feature_view, 
    model_air_quality,
    encoder,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=True,
)

print(response3)

Finished: Reading data from Hopsworks, using ArrowFlight (8.11s) 
🗓️ Today's date: Wednesday, 2024-03-27
📖 Air Quality Measurements for London:
Date: 2024-03-27; Air Quality: 6.4
Date: 2024-03-28; Air Quality: 9.77
Date: 2024-03-29; Air Quality: 8.71
Date: 2024-03-30; Air Quality: 8.24
Date: 2024-03-31; Air Quality: 8.57
Date: 2024-04-01; Air Quality: 8.66
Date: 2024-04-02; Air Quality: 8.18

The air quality in London on 2024-04-02 is expected to be at a level of 8.18. This is considered to be within the moderate range, which means it is safe for most people to go outside and engage in outdoor activities. However, sensitive individuals, such as those with respiratory or cardiovascular conditions, should take precautions and limit their exposure to air pollution.


In [17]:
QUESTION4 = "What will the air quality be like in Chicago tomorrow?"

response4 = generate_response(
    QUESTION4, 
    feature_view, 
    model_air_quality, 
    encoder,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=True,
)

print(response4)

Finished: Reading data from Hopsworks, using ArrowFlight (9.34s) 
🗓️ Today's date: Wednesday, 2024-03-27
📖 Air Quality Measurements for Chicago:
Date: 2024-03-27; Air Quality: 3.0
Date: 2024-03-28; Air Quality: 8.06

Based on the air quality measurements for Chicago, tomorrow's air quality is expected to be better than today. The air quality on 2024-03-28 is measured at 8.06, which is within the safe range for most people. It is generally safe to go outside and engage in outdoor activities, but people with respiratory issues may still want to take precautions.


In [18]:
QUESTION5 = "What will the air quality be like in London next Sunday?"

response5 = generate_response(
    QUESTION5, 
    feature_view, 
    model_air_quality, 
    encoder,
    model_llm, 
    tokenizer, 
    llm_chain,
    verbose=True,
)

print(response5)

Finished: Reading data from Hopsworks, using ArrowFlight (8.11s) 
🗓️ Today's date: Wednesday, 2024-03-27
📖 Air Quality Measurements for London:
Date: 2024-03-27; Air Quality: 6.4
Date: 2024-03-28; Air Quality: 9.77
Date: 2024-03-29; Air Quality: 8.71
Date: 2024-03-30; Air Quality: 8.24
Date: 2024-03-31; Air Quality: 8.57
Date: 2024-04-01; Air Quality: 8.66
Date: 2024-04-02; Air Quality: 8.18
Date: 2024-04-03; Air Quality: 8.18
Date: 2024-04-04; Air Quality: 8.18
Date: 2024-04-05; Air Quality: 8.18
Date: 2024-04-06; Air Quality: 8.18
Date: 2024-04-07; Air Quality: 8.18

Based on the air quality measurements for London, next Sunday, 2024-04-07, the air quality is expected to be 8.18. This level of air quality is considered safe, but it might be better to avoid strenuous outdoor activities if you have respiratory issues.


In [19]:
QUESTION7 = "What will the air quality be like on April 3 in London?"

response7 = generate_response(
    QUESTION7,
    feature_view,
    model_air_quality,
    encoder,
    model_llm,
    tokenizer,
    llm_chain,
    verbose=True,
)

print(response7)

Finished: Reading data from Hopsworks, using ArrowFlight (7.96s) 
🗓️ Today's date: Wednesday, 2024-03-27
📖 Air Quality Measurements for London:
Date: 2024-03-27; Air Quality: 6.4
Date: 2024-03-28; Air Quality: 9.77
Date: 2024-03-29; Air Quality: 8.71
Date: 2024-03-30; Air Quality: 8.24
Date: 2024-03-31; Air Quality: 8.57
Date: 2024-04-01; Air Quality: 8.66
Date: 2024-04-02; Air Quality: 8.18
Date: 2024-04-03; Air Quality: 8.18



The air quality on April 3 in London is expected to be safe for outdoor activities. The air quality index is around 8.18, which falls within the moderate range. This means that while the air may not be perfect, it is generally safe for most people to go outside and engage in physical activities.


In [20]:
QUESTION = "Can you please explain different PM2_5 air quality levels?"

response = generate_response(
    QUESTION, 
    feature_view, 
    model_air_quality, 
    encoder,
    model_llm, 
    tokenizer,
    llm_chain,
    verbose=True,
)

print(response)

🗓️ Today's date: Wednesday, 2024-03-27
📖 



Certainly! PM2.5 levels are categorized as follows:

1. Good (below 12 µg/m³): At this level, the air quality is considered safe for everyone, including those who are sensitive to air pollution. It's a great time to go for a walk or engage in outdoor activities.

2. Moderate (12-18 µg/m³): While the air quality is generally safe, people with respiratory issues or sensitivities may experience some discomfort. It's still suitable for outdoor activities, but those with breathing concerns should take precautions.

3. Poor (18-25 µg/m³): At this level, air quality is considered unhealthy for sensitive groups, such as children, the elderly, and those with respiratory conditions. It's advisable to limit prolonged outdoor exertion and consider indoor activities.

4. Very poor (25-35 µg/m³): The air quality is unhealthy for the general population, and it's advised to minimize outdoor activities, especially for sensitive groups.

5. Hazardous (above 35 µg/m³): At this level, air quality is cons

---

## <span style='color:#ff5f27'>🧬 Inference with OpenAI


In [21]:
from openai import OpenAI
import os
import getpass

from functions.llm_chain import generate_response_openai

In [22]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass.getpass('🔑 Enter your OpenAI API key: ')

client = OpenAI(
    api_key=os.environ["OPENAI_API_KEY"],
)

🔑 Enter your OpenAI API key:  ···················································


In [23]:
QUESTION = "What was the air quality like last week in London?"

response = generate_response_openai(   
    QUESTION,
    feature_view,
    model_air_quality,
    encoder,
    client,
    verbose=True,
)
print(response)

2024-03-27 14:42:13,003 INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Finished: Reading data from Hopsworks, using ArrowFlight (8.23s) 
🗓️ Today's date: Wednesday, 2024-03-27
📖 Air Quality Measurements for London:
Date: 2024-03-18; Air Quality: 12.7
Date: 2024-03-19; Air Quality: 9.7
Date: 2024-03-20; Air Quality: 15.6
Date: 2024-03-21; Air Quality: 16.7
Date: 2024-03-22; Air Quality: 8.7
Date: 2024-03-23; Air Quality: 5.4
Date: 2024-03-24; Air Quality: 6.4
2024-03-27 14:42:41,602 INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Last week in London, the air quality varied, starting off at a moderate level of 12.7 on the 18th. It slightly improved to 9.7 by the 19th, indicating relatively clean air that would be quite suitable for outdoor activities. There was a slight uptick in pollutants midweek, with air quality readings reaching 15.6 and 16.7 on the 20th and 21st respectively, suggesting a decrease in air q

In [24]:
QUESTION4 = "What will the air quality be like in Chicago tomorrow?"

response4 = generate_response_openai(
    QUESTION4,
    feature_view,
    model_air_quality,
    encoder,
    client,
    verbose=True,
)

print(response4)

2024-03-27 14:42:43,248 INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Finished: Reading data from Hopsworks, using ArrowFlight (9.63s) 
🗓️ Today's date: Wednesday, 2024-03-27
📖 Air Quality Measurements for Chicago:
Date: 2024-03-27; Air Quality: 3.0
Date: 2024-03-28; Air Quality: 8.06
2024-03-27 14:43:02,325 INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
The air quality in Chicago tomorrow is expected to be at 8.06, which indicates a moderate level of pollutants. It's still relatively safe for most people, but individuals who are especially sensitive to air pollution might want to limit their outdoor activities. It's a good day to keep an eye on any changes if you have respiratory conditions or other health concerns related to air quality.


---